# Decision Trees in Practice

In [2]:
import graphlab
import pandas as pd
import numpy as np
from __future__ import division

# Load LendingClub Dataset

In [3]:
loans = graphlab.SFrame("E:\\Machine Learning\\U.W\\Classification\\lending-club-data.gl/")
loans.save("E:\\Machine Learning\\U.W\\Classification\\lending-club-data.gl/", format="csv")
loans = pd.read_csv("E:\\Machine Learning\\U.W\\Classification\\lending-club-data.csv")

This non-commercial license of GraphLab Create for academic use is assigned to lxn1021@gmail.com and will expire on November 18, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Xiaoning\AppData\Local\Temp\graphlab_server_1558826010.log.0
c:\python27\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [7]:
loans.columns.values

array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'is_inc_v', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc',
       'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans',
       'bad_loans', 'emp_length_num', 'grade_num', '

In [9]:
loans["safe_loans"] = loans["bad_loans"].apply(lambda x: +1 if x==0 else -1)
loans = loans.drop(["bad_loans"], axis=1)

In [10]:
loans.columns.values

array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'is_inc_v', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc',
       'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans',
       'emp_length_num', 'grade_num', 'sub_grade_num

In [11]:
features = ["grade", "term", "home_ownership", "emp_length"]
target = "safe_loans"
loans = loans[features + [target]]

In [12]:
loans.head()

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1


## Subsample dataset to make sure classes are balanced

In [14]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans= safe_loans_raw.sample(frac=percentage, random_state=42)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                :", len(safe_loans)/float(len(loans_data))
print "Percentage of risky loans               :", len(risky_loans)/float(len(loans_data))
print "Total number of loans in our new dataset:", len(loans_data)

Percentage of safe loans                : 0.5
Percentage of risky loans               : 0.5
Total number of loans in our new dataset: 46300


## Transform categorical data into binary features

In [15]:
loans_data = risky_loans.append(safe_loans)

In [27]:
loans_data["emp_length"].value_counts()

10+ years    12744
2 years       4413
< 1 year      4210
3 years       3857
5 years       3630
4 years       3221
1 year        3190
6 years       2953
7 years       2581
8 years       2017
9 years       1681
Name: emp_length, dtype: int64

In [30]:
one_hot_grade = pd.get_dummies(loans_data["grade"], prefix="grade")
one_hot_term = pd.get_dummies(loans_data["term"], prefix="term")
one_hot_ownership = pd.get_dummies(loans_data["home_ownership"], prefix="home_ownership")
one_hot_emp = pd.get_dummies(loans_data["emp_length"], prefix="emp_length")

loans_data = pd.concat([one_hot_grade, one_hot_term, one_hot_ownership, one_hot_emp, loans_data["safe_loans"]], axis=1)

In [32]:
loans_data.head()

,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,...,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,safe_loans
1,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,-1
6,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,-1
7,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,-1
10,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,-1
12,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,-1


In [37]:
features = list(loans_data.columns.values)
features.remove("safe_loans")
features

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'term_ 36 months',
 'term_ 60 months',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'emp_length_1 year',
 'emp_length_10+ years',
 'emp_length_2 years',
 'emp_length_3 years',
 'emp_length_4 years',
 'emp_length_5 years',
 'emp_length_6 years',
 'emp_length_7 years',
 'emp_length_8 years',
 'emp_length_9 years',
 'emp_length_< 1 year']

## Train-Validation split

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
train_data, validation_set = train_test_split(loans_data, test_size=0.2, random_state=42)

In [46]:
print len(train_data)/float(len(loans_data))
print len(validation_set)/float(len(loans_data))

0.8
0.2


# Early stopping methods for decision trees

## Early stopping condition 1: Maximum depth

## Early stopping condition 2: Minimum node size

In [47]:
def reached_minimum_node_size(data, min_node_size):
    if len(data) <= min_node_size:
        return True
    else:
        return False

## Early stopping condition 3: Minimum gain in error reduction

In [48]:
def error_reduction(error_before_split, error_after_split):
    return error_before_split - error_after_split

## Grabbing binary decision tree helper functions from past assignment

In [49]:
def intermediate_node_num_mistakes(labels_in_node):
    if len(labels_in_node) == 0:
        return 0
    
    safe_loans = (labels_in_node == 1).sum()
    risky_loans = (labels_in_node == -1).sum()
    
    if safe_loans <= risky_loans:
        return safe_loans
    else:
        return risky_loans

In [50]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None
    best_error = 10
    
    num_data_points = float(len(data))
    
    for feature in features:
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] == 1]
        left_mistakes = intermediate_node_num_mistakes(left_split[target])
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
        
        error = (left_mistakes + right_mistakes) / num_data_points
        if error < best_error:
            best_feature = feature
            best_error = error
            
    
    return best_feature

In [51]:
def create_leaf(target_values):
    
    leaf = {"splitting_feature": None,
           "left": None,
           "right": None,
           "is_leaf": True}
    
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    if num_ones > num_minus_ones:
        leaf["prediction"] = +1
    else:
        leaf["prediction"] = -1
        
        
    return leaf

## Incorporating new early stopping conditions in binary decision tree implementation

In [53]:
def decision_tree_create(data, features, target, current_depth = 0,
                        max_depth = 10, min_node_size = 1,
                        min_error_reduction = 0.0):
    
    remaining_features = features[:]
    
    target_values = data[target]
    print "------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    
    if intermediate_node_num_mistakes(target_values) == 0:
        print "Stopping condition 1 reached. All data points have the same target value."
        return create_leaf(target_values)
    
    if remaining_features == []:
        print "Stopping condition 2 reached. No remaining features."
        return create_leaf(target_values)
    
    
    if current_depth >= max_depth:
        print "Early stopping condition 1 reached. Reached maximum depth."
        return create_leaf(target_values)
    
    if len(data) <= min_node_size:
        print "Early stopping condition 2 reached. Reached minimum node size."
        return create_leaf(target_values)
    
    
    splitting_feature = best_splitting_feature(data, features, target)
    
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    
    error_before_split = intermediate_node_num_mistakes(target_values)/float(len(data))
    
    left_mistakes = intermediate_node_num_mistakes(left_split[target])
    right_mistakes = intermediate_node_num_mistakes(right_split[target])
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    if error_before_split - error_after_split <= min_error_reduction:
        print "Early stopping condition 3 reached. Minimum error reduction."
        return create_leaf(target_values)
    
    
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (splitting_feature, len(left_split), len(right_split))
    
    
    left_tree = decision_tree_create(left_split, remaining_features, target,
                                    current_depth + 1, max_depth, min_node_size, min_error_reduction)
    
    right_tree = decision_tree_create(right_split, remaining_features, target,
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)
    
    
    return {"is_leaf": False,
           "prediction": None,
           "splitting_feature": splitting_feature,
           "left": left_tree,
           "right": right_tree}

In [54]:
def count_nodes(tree):
    if tree["is_leaf"]:
        return 1
    
    return 1 + count_nodes(tree["left"]) + count_nodes(tree["right"])

In [55]:
small_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 2, 
                                        min_node_size = 10, min_error_reduction=0.0)
if count_nodes(small_decision_tree) == 7:
    print 'Test passed!'
else:
    print 'Test failed... try again!'
    print 'Number of nodes found                :', count_nodes(small_decision_tree)
    print 'Number of nodes that should be there : 7' 

------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term_ 36 months. (9199, 27841)
------------------------------------------------------------------
Subtree, depth = 1 (9199 data points).
Split on feature grade_A. (9102, 97)
------------------------------------------------------------------
Subtree, depth = 2 (9102 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 2 (97 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 1 (27841 data points).
Split on feature grade_D. (23204, 4637)
------------------------------------------------------------------
Subtree, depth = 2 (23204 data points).
Early stopping condition 1 reached. Reached maximum depth.
-------------------------------------------------------------

## Build a tree!

In [56]:
my_decision_tree_new = decision_tree_create(train_data, features, "safe_loans", max_depth = 6,
                                           min_node_size = 100, min_error_reduction = 0.0)

------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term_ 36 months. (9199, 27841)
------------------------------------------------------------------
Subtree, depth = 1 (9199 data points).
Split on feature grade_A. (9102, 97)
------------------------------------------------------------------
Subtree, depth = 2 (9102 data points).
Early stopping condition 3 reached. Minimum error reduction.
------------------------------------------------------------------
Subtree, depth = 2 (97 data points).
Early stopping condition 2 reached. Reached minimum node size.
------------------------------------------------------------------
Subtree, depth = 1 (27841 data points).
Split on feature grade_D. (23204, 4637)
------------------------------------------------------------------
Subtree, depth = 2 (23204 data points).
Split on feature grade_E. (21896, 1308)
------------------------------------------------------------------
Subtree

In [57]:
my_decision_tree_old = decision_tree_create(train_data, features, "safe_loans", max_depth = 6,
                                           min_node_size = 0, min_error_reduction = -1)

------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term_ 36 months. (9199, 27841)
------------------------------------------------------------------
Subtree, depth = 1 (9199 data points).
Split on feature grade_A. (9102, 97)
------------------------------------------------------------------
Subtree, depth = 2 (9102 data points).
Split on feature grade_B. (8064, 1038)
------------------------------------------------------------------
Subtree, depth = 3 (8064 data points).
Split on feature grade_C. (5895, 2169)
------------------------------------------------------------------
Subtree, depth = 4 (5895 data points).
Split on feature grade_D. (3860, 2035)
------------------------------------------------------------------
Subtree, depth = 5 (3860 data points).
Split on feature home_ownership_OTHER. (3859, 1)
------------------------------------------------------------------
Subtree, depth = 6 (3859 data points).
Early 

Split on feature grade_C. (14370, 7148)
------------------------------------------------------------------
Subtree, depth = 5 (14370 data points).
Split on feature grade_G. (14264, 106)
------------------------------------------------------------------
Subtree, depth = 6 (14264 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 6 (106 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 5 (7148 data points).
Split on feature home_ownership_MORTGAGE. (4304, 2844)
------------------------------------------------------------------
Subtree, depth = 6 (4304 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 6 (2844 data points).
Early stopping condition 1 reached. Reached maximum 

## Making predictions

In [58]:
def classify(tree, x, annotate = False):
    if tree["is_leaf"]:
        if annotate:
            print "At leaf, predicting %s" % tree["prediction"]
        return tree["prediction"]
    else:
        split_feature_value = x[tree["splitting_feature"]]
        if annotate:
            print "Split on %s = %s" % (tree["splitting_feature"], split_feature_value)
        if split_feature_value == 0:
            return classify(tree["left"], x, annotate)
        else:
            return classify(tree["right"], x, annotate)

In [60]:
validation_set.iloc[0]

grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            1
term_ 60 months            0
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         0
emp_length_3 years         1
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
safe_loans                 1
Name: 53330, dtype: int64

In [63]:
print "Predicted class: %s" % classify(my_decision_tree_new, validation_set.iloc[0])

Predicted class: -1


In [64]:
classify(my_decision_tree_new, validation_set.iloc[0], annotate = True)

Split on term_ 36 months = 1
Split on grade_D = 1
At leaf, predicting -1


-1

In [65]:
classify(my_decision_tree_old, validation_set.iloc[0], annotate = True)

Split on term_ 36 months = 1
Split on grade_D = 1
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_E = 0
At leaf, predicting -1


-1

## Evaluating the model

In [68]:
def evaluate_classification_error(tree, data, target):
    prediction = []
    for i in range(len(data)):
        prediction.append(classify(tree, data.iloc[i]))
        
    error = (np.array(data[target]) != np.array(prediction)).sum()/float(len(data))
    
    
    return error

In [69]:
evaluate_classification_error(my_decision_tree_new, validation_set, target)

0.3769978401727862

In [70]:
evaluate_classification_error(my_decision_tree_old, validation_set, target)

0.3769978401727862

# Exploring the effect of max_depth

In [73]:
model_1 = decision_tree_create(train_data, features, "safe_loans", max_depth = 2,
                              min_node_size = 0, min_error_reduction = -1)

model_2 = decision_tree_create(train_data, features, "safe_loans", max_depth = 6,
                              min_node_size = 0, min_error_reduction = -1)

model_3 = decision_tree_create(train_data, features, "safe_loans", max_depth = 14,
                              min_node_size = 0, min_error_reduction = -1)

------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term_ 36 months. (9199, 27841)
------------------------------------------------------------------
Subtree, depth = 1 (9199 data points).
Split on feature grade_A. (9102, 97)
------------------------------------------------------------------
Subtree, depth = 2 (9102 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 2 (97 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 1 (27841 data points).
Split on feature grade_D. (23204, 4637)
------------------------------------------------------------------
Subtree, depth = 2 (23204 data points).
Early stopping condition 1 reached. Reached maximum depth.
-------------------------------------------------------------

Split on feature grade_B. (3, 0)
------------------------------------------------------------------
Subtree, depth = 5 (3 data points).
Split on feature grade_C. (3, 0)
------------------------------------------------------------------
Subtree, depth = 6 (3 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 6 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 5 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 1 (27841 data points).
Split on feature grade_D. (23204, 4637)
------------------------------------------------------------------
Subtree, depth = 2 (23204 data points).
Split on feature grade_E. (21896, 1308)
---------------------

Subtree, depth = 12 (35 data points).
Split on feature emp_length_8 years. (34, 1)
------------------------------------------------------------------
Subtree, depth = 13 (34 data points).
Split on feature home_ownership_RENT. (34, 0)
------------------------------------------------------------------
Subtree, depth = 14 (34 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 14 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 13 (1 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 11 (152 data points).
Split on feature home_ownership_OWN. (152, 0)
------------------------------------------------------------------
Subtree, depth = 12 (152 da

Split on feature grade_E. (2035, 0)
------------------------------------------------------------------
Subtree, depth = 6 (2035 data points).
Split on feature grade_F. (2035, 0)
------------------------------------------------------------------
Subtree, depth = 7 (2035 data points).
Split on feature grade_G. (2035, 0)
------------------------------------------------------------------
Subtree, depth = 8 (2035 data points).
Split on feature term_ 60 months. (0, 2035)
------------------------------------------------------------------
Subtree, depth = 9 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 9 (2035 data points).
Split on feature home_ownership_MORTGAGE. (916, 1119)
------------------------------------------------------------------
Subtree, depth = 10 (916 data points).
Split on feature home_ownership_OTHER. (915, 1)
---------------------------------------

Split on feature home_ownership_OWN. (1357, 0)
------------------------------------------------------------------
Subtree, depth = 12 (1357 data points).
Split on feature home_ownership_RENT. (1357, 0)
------------------------------------------------------------------
Subtree, depth = 13 (1357 data points).
Split on feature emp_length_1 year. (1301, 56)
------------------------------------------------------------------
Subtree, depth = 14 (1301 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 14 (56 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 13 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 12 (0 data points).
Stopping condition 1 reached. Al

...

------------------------------------------------------------------
Subtree, depth = 10 (55 data points).
Split on feature emp_length_6 years. (54, 1)
------------------------------------------------------------------
Subtree, depth = 11 (54 data points).
Split on feature home_ownership_OTHER. (54, 0)
------------------------------------------------------------------
Subtree, depth = 12 (54 data points).
Split on feature home_ownership_OWN. (54, 0)
------------------------------------------------------------------
Subtree, depth = 13 (54 data points).
Split on feature home_ownership_RENT. (54, 0)
------------------------------------------------------------------
Subtree, depth = 14 (54 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 14 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
-------------------------------------------------------

Split on feature emp_length_1 year. (2688, 156)
------------------------------------------------------------------
Subtree, depth = 7 (2688 data points).
Split on feature emp_length_6 years. (2488, 200)
------------------------------------------------------------------
Subtree, depth = 8 (2488 data points).
Split on feature emp_length_3 years. (2294, 194)
------------------------------------------------------------------
Subtree, depth = 9 (2294 data points).
Split on feature grade_A. (2294, 0)
------------------------------------------------------------------
Subtree, depth = 10 (2294 data points).
Split on feature grade_B. (2294, 0)
------------------------------------------------------------------
Subtree, depth = 11 (2294 data points).
Split on feature grade_G. (2294, 0)
------------------------------------------------------------------
Subtree, depth = 12 (2294 data points).
Split on feature term_ 60 months. (2294, 0)
---------------------------------------------------------------

Split on feature grade_B. (369, 0)
------------------------------------------------------------------
Subtree, depth = 7 (369 data points).
Split on feature grade_C. (369, 0)
------------------------------------------------------------------
Subtree, depth = 8 (369 data points).
Split on feature grade_G. (369, 0)
------------------------------------------------------------------
Subtree, depth = 9 (369 data points).
Split on feature term_ 60 months. (369, 0)
------------------------------------------------------------------
Subtree, depth = 10 (369 data points).
Split on feature home_ownership_MORTGAGE. (238, 131)
------------------------------------------------------------------
Subtree, depth = 11 (238 data points).
Split on feature home_ownership_OTHER. (234, 4)
------------------------------------------------------------------
Subtree, depth = 12 (234 data points).
Split on feature home_ownership_OWN. (205, 29)
------------------------------------------------------------------
Subt

------------------------------------------------------------------
Subtree, depth = 13 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 12 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 10 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 9 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 8 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 7 (0 data points).
Stopping 

Split on feature home_ownership_OWN. (32, 0)
------------------------------------------------------------------
Subtree, depth = 13 (32 data points).
Split on feature home_ownership_RENT. (32, 0)
------------------------------------------------------------------
Subtree, depth = 14 (32 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 14 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 13 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 12 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
------------------------------------------------------------------
Subtree, depth = 9 (0 data points).
Sto

### Evaluating the models

In [76]:
print "Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data, target)
print "Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data, target)
print "Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data, target)

Training data, classification error (model 1): 0.398110151187905
Training data, classification error (model 2): 0.3803995680345572
Training data, classification error (model 3): 0.37583693304535637


In [77]:
print "Validation data, classification error (model 1):", evaluate_classification_error(model_1, validation_set, target)
print "Validation data, classification error (model 2):", evaluate_classification_error(model_2, validation_set, target)
print "Validation data, classification error (model 3):", evaluate_classification_error(model_3, validation_set, target)

Validation data, classification error (model 1): 0.39773218142548594
Validation data, classification error (model 2): 0.3769978401727862
Validation data, classification error (model 3): 0.38196544276457883


### Measuring the complexity of the tree

In [78]:
def count_leaves(tree):
    if tree["is_leaf"]:
        return 1
    
    return count_leaves(tree["left"]) + count_leaves(tree["right"])

In [79]:
print "Number of nodes (model 1):", count_leaves(model_1)
print "Number of nodes (model 2):", count_leaves(model_2)
print "Number of nodes (model 3):", count_leaves(model_3)

Number of nodes (model 1): 4
Number of nodes (model 2): 42
Number of nodes (model 3): 361


# Exploring the effect of min_error

In [80]:
model_4 = decision_tree_create(train_data, features, "safe_loans", max_depth = 6,
                              min_node_size = 0, min_error_reduction = -1)

model_5 = decision_tree_create(train_data, features, "safe_loans", max_depth = 6,
                              min_node_size = 0, min_error_reduction = 0)

model_6 = decision_tree_create(train_data, features, "safe_loans", max_depth = 6,
                              min_node_size = 0, min_error_reduction = 5)

------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term_ 36 months. (9199, 27841)
------------------------------------------------------------------
Subtree, depth = 1 (9199 data points).
Split on feature grade_A. (9102, 97)
------------------------------------------------------------------
Subtree, depth = 2 (9102 data points).
Split on feature grade_B. (8064, 1038)
------------------------------------------------------------------
Subtree, depth = 3 (8064 data points).
Split on feature grade_C. (5895, 2169)
------------------------------------------------------------------
Subtree, depth = 4 (5895 data points).
Split on feature grade_D. (3860, 2035)
------------------------------------------------------------------
Subtree, depth = 5 (3860 data points).
Split on feature home_ownership_OTHER. (3859, 1)
------------------------------------------------------------------
Subtree, depth = 6 (3859 data points).
Early 

Split on feature grade_F. (21518, 378)
------------------------------------------------------------------
Subtree, depth = 4 (21518 data points).
Split on feature grade_C. (14370, 7148)
------------------------------------------------------------------
Subtree, depth = 5 (14370 data points).
Split on feature grade_G. (14264, 106)
------------------------------------------------------------------
Subtree, depth = 6 (14264 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 6 (106 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 5 (7148 data points).
Split on feature home_ownership_MORTGAGE. (4304, 2844)
------------------------------------------------------------------
Subtree, depth = 6 (4304 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------

Split on feature emp_length_8 years. (369, 9)
------------------------------------------------------------------
Subtree, depth = 5 (369 data points).
Early stopping condition 3 reached. Minimum error reduction.
------------------------------------------------------------------
Subtree, depth = 5 (9 data points).
Early stopping condition 3 reached. Minimum error reduction.
------------------------------------------------------------------
Subtree, depth = 3 (1308 data points).
Early stopping condition 3 reached. Minimum error reduction.
------------------------------------------------------------------
Subtree, depth = 2 (4637 data points).
Early stopping condition 3 reached. Minimum error reduction.
------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Early stopping condition 3 reached. Minimum error reduction.


In [83]:
print "Validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_set, target)
print "Validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_set, target)
print "Validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_set, target)

Validation data, classification error (model 4): 0.3769978401727862
Validation data, classification error (model 5): 0.37688984881209503
Validation data, classification error (model 6): 0.5001079913606912


In [84]:
print "Number of nodes (model 4):", count_leaves(model_4)
print "Number of nodes (model 5):", count_leaves(model_5)
print "Number of nodes (model 6):", count_leaves(model_6)

Number of nodes (model 4): 42
Number of nodes (model 5): 15
Number of nodes (model 6): 1


# Exploring the effect of min_node_size

In [85]:
model_7 = decision_tree_create(train_data, features, "safe_loans", max_depth = 6,
                              min_node_size = 0, min_error_reduction = -1)

model_8 = decision_tree_create(train_data, features, "safe_loans", max_depth = 6,
                              min_node_size = 2000, min_error_reduction = -1)

model_9 = decision_tree_create(train_data, features, "safe_loans", max_depth = 6,
                              min_node_size = 50000, min_error_reduction = -1)

------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term_ 36 months. (9199, 27841)
------------------------------------------------------------------
Subtree, depth = 1 (9199 data points).
Split on feature grade_A. (9102, 97)
------------------------------------------------------------------
Subtree, depth = 2 (9102 data points).
Split on feature grade_B. (8064, 1038)
------------------------------------------------------------------
Subtree, depth = 3 (8064 data points).
Split on feature grade_C. (5895, 2169)
------------------------------------------------------------------
Subtree, depth = 4 (5895 data points).
Split on feature grade_D. (3860, 2035)
------------------------------------------------------------------
Subtree, depth = 5 (3860 data points).
Split on feature home_ownership_OTHER. (3859, 1)
------------------------------------------------------------------
Subtree, depth = 6 (3859 data points).
Early 

Split on feature grade_F. (21518, 378)
------------------------------------------------------------------
Subtree, depth = 4 (21518 data points).
Split on feature grade_C. (14370, 7148)
------------------------------------------------------------------
Subtree, depth = 5 (14370 data points).
Split on feature grade_G. (14264, 106)
------------------------------------------------------------------
Subtree, depth = 6 (14264 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 6 (106 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 5 (7148 data points).
Split on feature home_ownership_MORTGAGE. (4304, 2844)
------------------------------------------------------------------
Subtree, depth = 6 (4304 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------

Split on feature grade_G. (14264, 106)
------------------------------------------------------------------
Subtree, depth = 6 (14264 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 6 (106 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 5 (7148 data points).
Split on feature home_ownership_MORTGAGE. (4304, 2844)
------------------------------------------------------------------
Subtree, depth = 6 (4304 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 6 (2844 data points).
Early stopping condition 1 reached. Reached maximum depth.
------------------------------------------------------------------
Subtree, depth = 4 (378 data points).
Early stopping condition 2 reached.

In [86]:
print "Validation data, classification error (model 7):", evaluate_classification_error(model_7, validation_set, target)
print "Validation data, classification error (model 8):", evaluate_classification_error(model_8, validation_set, target)
print "Validation data, classification error (model 9):", evaluate_classification_error(model_9, validation_set, target)

Validation data, classification error (model 7): 0.3769978401727862
Validation data, classification error (model 8): 0.37667386609071274
Validation data, classification error (model 9): 0.5001079913606912


In [87]:
print "Number of nodes (model 7):", count_leaves(model_7)
print "Number of nodes (model 8):", count_leaves(model_8)
print "Number of nodes (model 9):", count_leaves(model_9)

Number of nodes (model 7): 42
Number of nodes (model 8): 20
Number of nodes (model 9): 1
